# Putting it all together

### @binance, @justinsuntron, @rogerkver, @cz_binance, @brian_armstrong, @NickSzabo4, @ethereum,
### @blockchain, @CoinDesk, @BitcoinMagazine, @VitalikButerin

In [1]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
# from msedge.selenium_tools import Edge, EdgeOptions
from selenium import webdriver

def get_tweet_data(card):
    """Extract data from tweet data"""
    username = card.find_element_by_xpath('.//span').text
    handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return 
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet

# tweet_list = ["binance", "justinsuntron", "rogerkver", "cz_binance", "brian_armstrong", "NickSzabo4", "ethereum", 
#              "blockchain", "CoinDesk", "BitcoinMagazine"]
tweet_list = ['VitalikButerin']


driver = webdriver.Chrome('C:/Users/choi/chromedriver')

# 암묵적으로 웹 자원 로드를 위해 3초까지 기다려 준다.
driver.implicitly_wait(3)
driver.get('https://www.twitter.com/login')
driver.maximize_window()

username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys('sparrow42@naver.com')

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys('1234qwerasdf')
password.send_keys(Keys.RETURN)

for tw in tweet_list:
    print(tw, ' 데이터 수집 시작')
    # find search input and search for term
    driver.get('https://www.twitter.com/'+tw)

    # get all tweets on the page
    data = []
    tweet_ids = set()
    last_position = driver.execute_script("return window.pageYOffset;")
    scrolling = True

    while scrolling:
        page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
        for card in page_cards[-15:]:
            tweet = get_tweet_data(card)
            if tweet:
                tweet_id = ''.join(tweet)
                if tweet_id not in tweet_ids:
                    tweet_ids.add(tweet_id)
                    data.append(tweet)

        scroll_attempt = 0
        while True:
            # check scroll position
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(1)
            curr_position = driver.execute_script("return window.pageYOffset;")
            if last_position == curr_position:
                scroll_attempt += 1

                # end of scroll region
                if scroll_attempt >=  3:
                    scrolling = False
                    break
                else:
                    sleep(2) # attempt to scroll again

            else:
                last_position = curr_position
                break

    print(tw+' 길이 : ', len(data))

    # 파일 쓰기
    with open(tw+'_tweets.csv', 'w', newline='', encoding='utf-8') as f:
        header = ['UserName', 'Handle', 'Timestamp', 'Comments', 'Likes', 'Retweets', 'Text']
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(data)

VitalikButerin  데이터 수집 시작
VitalikButerin 길이 :  429
